## Method 1



In [78]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt

rho_oil = 875.3
rho_air = 1.204
g = 9.80
nu = 1.827e-5
d = 6.0e-3
precision = 19

velocities_file = '/Users/lucaschoi/Documents/GitHub/PHY294-Milikan-Oil-Drop-Experiment/velocities.tsv'
method1_results_file = '/Users/lucaschoi/Documents/GitHub/PHY294-Milikan-Oil-Drop-Experiment/method1.py'

charges = []
charge_uncs = []
V_unc = 0.3

velocities_df = pd.read_csv(velocities_file, sep='\t')



In [79]:
for _, row in velocities_df.iterrows():
    try:
        V_stop = row['stopping_voltage']
        v_d = row['v_fall']
        v_d_unc = row['v_fall_unc']

        q = v_d ** (3/2) / V_stop

        # Calculate the uncertainty in charge
        partial_q_vd = (3 / 2) * (v_d ** (1/2)) / V_stop
        partial_q_V_stop = -(v_d ** (3/2)) / (V_stop ** 2)
        q_unc = np.sqrt((partial_q_vd * v_d_unc) ** 2 + (partial_q_V_stop * V_unc) ** 2)
        charges.append(q)
        charge_uncs.append(q_unc)
        # print(f'q = {q:.10} \pm {q_unc:.10} C')

    except Exception as e:
        print(f"{e}")
all_e = []

In [87]:
def estimate_elementary_charge(q_list, e_min=1e-19, e_max=2e-19, num_steps=1000000):
    """
    Estimate the elementary charge by trying possible divisors of q_list.
    
    Args:
        q_list (array-like): Measured charges in Coulombs
        e_min, e_max: Range to scan for candidate e
        num_steps: Number of candidate e values to test
    
    Returns:
        best_e: Estimated elementary charge
        scores: List of scores for all candidate e values
    """
    q_array = np.array(q_list)
    candidate_es = np.linspace(e_min, e_max, num_steps)
    best_score = float('inf')
    best_e = None
    scores = []

    for e in candidate_es:

        # Divide all q values by e and check how close to nearest integer
        multiples = q_array / e
        residuals = np.abs(multiples - np.round(multiples))
        score = np.mean(residuals) 
        scores.append(score)
        all_e.append((e, score))
        
        if score < best_score:
            best_score = score
            best_e = e

    return best_e, scores, candidate_es

In [101]:
# sort the all_e's by the best score
all_e.sort(key=lambda x: x[1])

for i in range(100):
    print(f"Best e = {all_e[i][0]:.3e} C with score {all_e[i][1]}")

# average of best 10 ** n
for i in range(5):

    scale = 10 ** i
    if scale > len(all_e):
        break
    print(f"Average of best {scale} = {np.mean([all_e[i][0] for i in range(scale)]):.3e} C")



Best e = 1.863e-19 C with score 0.16902934803682215
Best e = 1.863e-19 C with score 0.16902934803682215
Best e = 1.509e-19 C with score 0.17386960048301547
Best e = 1.862e-19 C with score 0.17412273556578392
Best e = 1.862e-19 C with score 0.17412273556578392
Best e = 1.517e-19 C with score 0.17462911792829924
Best e = 1.863e-19 C with score 0.1747447088652966
Best e = 1.863e-19 C with score 0.1747447088652966
Best e = 1.861e-19 C with score 0.1754018185185451
Best e = 1.861e-19 C with score 0.1754018185185451
Best e = 1.861e-19 C with score 0.17690764221490599
Best e = 1.861e-19 C with score 0.17690764221490599
Best e = 1.863e-19 C with score 0.17829123665304744
Best e = 1.863e-19 C with score 0.17829123665304744
Best e = 1.517e-19 C with score 0.1790781021118164
Best e = 1.861e-19 C with score 0.17978852402930165
Best e = 1.861e-19 C with score 0.17978852402930165
Best e = 1.863e-19 C with score 0.18045810624664904
Best e = 1.863e-19 C with score 0.18045810624664904
Best e = 1.863e-1